<a href="https://colab.research.google.com/github/Dumi-coder/CeylonPulse/blob/apiDum/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd

# --- 1. ASPI: Scraping TradingEconomics (Best effort) ---
def fetch_aspi_data():
    """Fetches ASPI data (Value, Change, %Change) from TradingEconomics."""
    url = "https://tradingeconomics.com/sri-lanka/stock-market"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    aspi_data = {"Value": "N/A", "Change": "N/A", "Pct": "N/A"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the main table containing the indices
        index_table = soup.find('table', {'class': 'table-hover'})
        if not index_table:
             raise Exception("Index table not found.")

        # Iterate through table rows
        for row in index_table.find_all("tr"):
            cols = row.find_all("td")

            # Check if the row is for ASPI or All Share
            if len(cols) > 0 and ("ASPI" in cols[0].get_text() or "All Share" in cols[0].get_text()):
                # Extracting data based on expected column position (Value, Change, Pct)
                # The data is usually in columns 1, 2, and 3

                # Check for sufficient columns before accessing
                if len(cols) > 1:
                    aspi_data["Value"] = cols[1].get_text().strip()
                if len(cols) > 2:
                    aspi_data["Change"] = cols[2].get_text().strip()
                if len(cols) > 3:
                    aspi_data["Pct"] = cols[3].get_text().strip()

                break
    except Exception as e:
        aspi_data["Error"] = f"ASPI Scraper failed: {e}"

    return aspi_data

# --- 2. S&P SL20: Using yfinance API (Guaranteed reliable data) ---
def fetch_sl20_data():
    """Fetches S&P SL20 data (Value, Change, %Change) from Yahoo Finance API."""
    ticker_symbol = '^SPLK20LP'
    sl20_data = {"Value": "N/A", "Change": "N/A", "Pct": "N/A"}

    try:
        ticker = yf.Ticker(ticker_symbol)
        data = ticker.info

        current_value = data.get('regularMarketPrice')
        change_amount = data.get('regularMarketChange')
        change_pct = data.get('regularMarketChangePercent')

        # Format the numbers
        sl20_data["Value"] = f"{current_value:,.2f}" if current_value is not None else "N/A"

        if change_amount is not None:
            sign = '+' if change_amount >= 0 else ''
            sl20_data["Change"] = f"{sign}{change_amount:,.2f}"
            sl20_data["Pct"] = f"{sign}{change_pct:.2f}%"
        else:
            sl20_data["Change"] = "N/A"
            sl20_data["Pct"] = "N/A"

    except Exception as e:
        sl20_data["Error"] = f"SL20 API failed: {e}"

    return sl20_data

# --- Main Execution and Output ---

aspi_result = fetch_aspi_data()
sl20_result = fetch_sl20_data()

print("\n" + "="*20)
print(" CSE Market Indices")
print("="*20)

# ASPI Output
print(" ASPI ")
print(f" {aspi_result['Value']} ")
print(f" {aspi_result['Change']} ")
print(f" {aspi_result['Pct']} ")

if 'Error' in aspi_result:
    print(f">>> ASPI Note: {aspi_result['Error']}")

print("-" * 20)

# S&P SL20 Output
print(" S&P SL20 ")
print(f" {sl20_result['Value']} ")
print(f" {sl20_result['Change']} ")
print(f" {sl20_result['Pct']} ")

if 'Error' in sl20_result:
    print(f">>> SL20 Note: {sl20_result['Error']}")

print("="*20)


 CSE Market Indices
 ASPI 
 21,497.08 
  
 -329.51 
--------------------
 S&P SL20 
 5,925.63 
 -105.03 
 -1.74% 
